In [1]:
import torch

In [2]:
%load_ext autoreload

%autoreload 3


In [3]:
from datasets import load_dataset

In [4]:
# for grammar correction
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
model_name="t5-small"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [7]:
from datasets import load_dataset

dataset = load_dataset("jfleg")

Using custom data configuration default
Reusing dataset jfleg (/home/ubuntu/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 755
    })
    test: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 748
    })
})

In [34]:
dataset_name = "grammar_validation.csv"

dataset = load_dataset("csv", data_files={"eval": dataset_name }, delimiter=",")

Using custom data configuration default-4dfaea94d7b521ea
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-4dfaea94d7b521ea/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
dataset_name = "grammar_train.csv"

dataset = load_dataset("csv", data_files={"train": dataset_name }, delimiter=",")

Using custom data configuration default-6dcaf22609f32d24
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-6dcaf22609f32d24/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
dataset.cleanup_cache_files()


{'eval': 0}

In [62]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 3016
    })
})


In [64]:
tokenized_dataset = dataset.map(
            _preprocess_samples,
            batched=True,
            num_proc=4,
            remove_columns=["input", "target"],
        )

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


In [65]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3016
    })
})

In [75]:
tokenized_dataset.column_names

{'train': ['input_ids', 'attention_mask', 'labels']}

In [67]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [68]:
# Pads inputs and labels to max length
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [69]:
data_collator

DataCollatorForSeq2Seq(tokenizer=PreTrainedTokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<ext

In [63]:
def _preprocess_samples(examples, max_len=1024):
        """
        
        """
        model_inputs = tokenizer(examples["input"], max_length=max_len, truncation=True)

        # Setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(examples["target"], max_length=max_len, truncation=True)

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

In [14]:
tokenized_dataset = dataset.map(
            _preprocess_samples(tokenizer=tokenizer),
            batched=True,
            num_proc=4,
            remove_columns=["input", "target"],
        )

TypeError: _preprocess_samples() missing 1 required positional argument: 'examples'

In [49]:
from jfleg_dataset import get_dataset

In [50]:
r = get_dataset(tokenizer, None, None)

Using custom data configuration default-591f2a430d3f1022
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-591f2a430d3f1022/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [51]:
r.dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 3016
    })
})

In [30]:
x = load_dataset("csv", data_files={"jfleg_validation.csv"}, delimiter=",")

Using custom data configuration default-93767cda0df62401


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/csv/default-93767cda0df62401/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [42]:
y = x["train"]
y.num_rows

2988

In [9]:
train_dataset = load_dataset("jfleg", split='validation[:]')

Using custom data configuration default
Reusing dataset jfleg (/home/ubuntu/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b)


In [11]:
train_dataset[100]

{'sentence': "It 's normal : propose something to the client and construct something different , is the best way ( and it 's legal ! ! ! ! ! ! ! ! ! ) ",
 'corrections': ["It 's normal , propose something to the client and make something different , it 's the best way and it 's legal ! ",
  "It 's normal : proposing something to the client and constructing something differently is the best way ( and it 's legal ! ) . ",
  "It 's normal : propose something to the client and construct something different , it is the best way and it is legal . ",
  "It 's normal : propose something to the client and construct something different , it is the best way ( and it 's legal ! ! ! ! ! ! ! ! ! ) . "]}

In [5]:
eval_dataset = load_dataset("jfleg", split='test[:]')

Using custom data configuration default
Reusing dataset jfleg (/home/ubuntu/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b)


In [6]:
for case in train_dataset["corrections"][5:9]:
  print(case)
  print(case[0])
  print("--------------------------------------------------------")

['They draw in the consumers , like me , to purchase a great product with amazing ingredients , but actually , they just make up these words to increase their sales . ', 'They draw in consumers like me , who purchase this great product with all these amazing ingredients , but sometimes they just make things up to increase their sales . ', 'They draw the consumers , like me , to purchase this great product with all these amazing features , but actually sometimes they just make something up just to increase their sales . ', 'They draw the consumers , like me , to purchase this great product with all these amazing ingredients and all that , but actually they just sometimes make something up just to increase their sales . ']
They draw in the consumers , like me , to purchase a great product with amazing ingredients , but actually , they just make up these words to increase their sales . 
--------------------------------------------------------
['I want to talk about nocive or bad products 

In [7]:
import csv

In [32]:
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases are blank strings. So we'll skip them
                #input_text = remove_excess_spaces(input_text)
                correction = remove_excess_spaces(correction)
                input_text = remove_excess_spaces(input_text)


                if input_text and correction:
                    writter.writerow([input_text, correction])

In [9]:
from pathlib import Path

In [10]:
data_dir = Path.cwd()/'datasets/jfleg'

In [33]:
generate_csv("jfleg_train.csv", train_dataset)

In [34]:
generate_csv("jfleg_validation.csv", eval_dataset)

In [12]:
replacements = [
  (" .", "."), 
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water')
]

def remove_excess_spaces(text):
  for rep in replacements:
    text = text.replace(rep[0], rep[1])

  return text


In [14]:
import pandas as pd

In [20]:
df = pd.read_csv('jfleg_train.csv', index_col=0, header=None)

In [22]:
df = df.iloc[1: , :]  # drop first row which has titles

In [17]:
z = df.to_dict()

In [23]:
df.head

<bound method NDFrame.head of                                                                                                     1
0                                                                                                    
grammar: So I think we can not live if old peop...  So I think we would not be alive if our ancest...
grammar: So I think we can not live if old peop...  So I think we could not live if older people d...
grammar: So I think we can not live if old peop...  So I think we can not live if old people could...
grammar: So I think we can not live if old peop...  So I think we can not live if old people can n...
grammar: For not use car .                                                  Not for use with a car . 
...                                                                                               ...
grammar: Sceene of violence can affect on them .                Scenes of violence can affect them . 
grammar: While the communities in general have ...  

In [24]:
z = df.to_dict()

In [26]:
type(z)

dict

In [28]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-small")

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [31]:

text = "I am lived with my parenmts "
print(f"untrained input: {text}")
inputs = tokenizer("grammar:"+text, truncation=True, return_tensors='pt')
print(f"inputes = {inputs}")
output = model.generate(inputs['input_ids'], num_beams=4, max_length=512, early_stopping=True)
correction=tokenizer.batch_decode(output, skip_special_tokens=True)
print("".join(correction)) #Correction: I am living with my parents.


untrained input: I am lived with my parenmts 
Ignored unknown kwarg option direction
inputes = {'input_ids': tensor([[19519,    10,   196,   183,  4114,    28,    82,   260,    35,    51,
            17,     7,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Grammatik:I am lived with my parenmts with my parenmts.
